# Preprocess GCM SIA Series

In [1]:
import xarray as xr
import numpy as np
import os
import pandas as pd
import scipy
from scipy import interpolate
import geopandas as gpd
import shapely.vectorized
from itertools import product

### Function to find, read, and concatenate data.

In [12]:
#GCM FUNCTIONS
def yearcheck(file, year):
    check = True
    with xr.open_dataset(file) as ds:
        time = ds.coords['time'].values[:]
        if type(time[0]) == np.datetime64:
            time = pd.to_datetime(time) 
        ds_years= [time[i].year for i in range(len(time))]
    if year not in ds_years:
        check = False
    return check

def extract_gcm_data(ncfile,var,year,month):
    with xr.open_dataset(ncfile) as ds:
        ds = ds.where(((ds['time.year'] == year)), drop=True)
        ds = ds.sel(time=ds.time.dt.month.isin([month]))
        dt = np.array(ds.coords['time'][:])
        if 'lat' in ds.variables and 'lon' in ds.variables:
            x,y = np.array(ds.variables['lon'][:]),np.array(ds.variables['lat'][:])
        elif 'latitude' in ds.variables and 'longitude' in ds.variables:
            x,y = np.array(ds.variables['longitude'][:]),np.array(ds.variables['latitude'][:])
        else:
            raise ValueError("Unable to find latitude and longitude variables in the dataset.")
        x = np.where(x<0,x+360,x)
        var_field = np.array(ds.variables[var][:])
    return var_field,x,y,dt

#Interpolate 3D GCM data to another grid
def gcm_interp(x,y,z,xgrid,ygrid):
    x = np.ravel(x)
    y = np.ravel(y)
    if np.ndim(z) == 3:
        for tstep in range(z.shape[0]):
            ztemp = np.ravel(z[tstep,:,:])
            temp = scipy.interpolate.griddata((x,y),ztemp,(xgrid,ygrid),method='nearest')

            if tstep == 0:
                zgrid = temp
            else:
                zgrid = np.dstack((zgrid,temp))
        zgrid = np.transpose(zgrid, (2, 0, 1))
    else:
        ztemp = np.ravel(z[:,:])
        temp = scipy.interpolate.griddata((x,y),ztemp,(xgrid,ygrid),method='linear')
        temp1 = scipy.interpolate.griddata((x,y),ztemp,(xgrid,ygrid),method='nearest')
        temp  = np.where(np.isnan(temp),temp1,temp)
        zgrid = temp
    return zgrid

def concatenate_gcm(rootdir,var,year0,yearn,month):
    years = np.arange(year0,yearn,1)
    for count,year in enumerate(years):
        file = find_gcm_file(rootdir,year)
        temp,x,y,temp_dt = extract_gcm_data(file,var,year,month)
        if count == 0:
            series = temp
            dt = temp_dt
        else:
            series = np.vstack((series,temp))
            dt = np.append(dt,temp_dt)
    return series,dt,x,y

def find_gcm_file(root_directory,year):
    for root, dirs, files in os.walk(root_directory):
        filecheck = False
        for file in files:
            filecheck = yearcheck(os.path.join(root, file),year)
            if filecheck == False:
                continue
            else:
                matching_file = (os.path.join(root, file))
                break
        if filecheck == False:
            matching_file = []
            print(f'A file which contained the year {year} was not found!')
    return matching_file

def OpenAreacello(root_directory):
    for root, dirs, files in os.walk(root_directory):
        for file in files:
            ncpath = os.path.join(root, file)
            with xr.open_dataset(ncpath) as ds:
                var_field = np.array(ds.variables['areacello'][:])                
    return var_field

## Create SIA derivation field function

In [82]:
def SIAFields(sic,areacello):
    #If 0 to 100 scale, transform to 0 to 1
    if np.nanmax(sic)>1.1:
        sic = sic/100
    sia = sic * areacello[np.newaxis,:,:]
    sia = sia/((10**6)*(10**6))
    return sia

## Extract Data Within Regional Shapefile

In [83]:
def sea_aves(field,seas,x,y):
    sea_names = []
    if np.nanmax(x)>180:
        x = np.where(x>180,x-360,x)
    for index, row in seas.iterrows():
        name = row['name']
        sea_names.append(name)
        poly = row['geometry']
        nodes= np.where(shapely.vectorized.contains(poly, x, y))
        sea_nodes = field[:,nodes[0],nodes[1]]
        if index == 0:
            sea_sum = np.nansum(sea_nodes,axis = 1)
        else:
            temp = np.nansum(sea_nodes,axis =1)
            sea_sum = np.vstack((sea_sum,temp))      
    return sea_sum,sea_names

# Derive the timeseries interating over models

In [88]:
months = [7,9,11]
year0 = 2020
yearn = 2070

fn = 'Regions.shp'
seas = gpd.GeoDataFrame.from_file(fn)
for model_name in ['CNRM','ECEARTH','MPI','MRI']:
    for i,month in enumerate(months):
        rootdir = f'/Sea Ice/{model_name}/siconc'
        sic,dt,x,y = concatenate_gcm(rootdir,'siconc',year0,yearn,month)
        rootdir = f'/Sea Ice/{model_name}/areacello'
        areacello = OpenAreacello(rootdir)
        sia = SIAFields(sic,areacello)
        sia_temp,regions = sea_aves(sia,seas,x,y)
        if i == 0:
            seas_sia = sia_temp
        else:
            seas_sia = np.vstack((seas_sia,sia_temp))
    
    months = ['July','September','November']
    cols = [x+str(' ')+y for (x,y) in product(months,regions)]
        
    df = pd.DataFrame(seas_sia.T, columns=cols)
    years = np.arange(year0,yearn,1)
    df.insert(0, ('year', ''), years)
    df.to_csv(f'{model_name}_SIA_series.csv', index=False)
    print("CSV file saved successfully.")

CSV file saved successfully.


### Filled Maximum Arrays for Thresholds

In [93]:
month = 9
year0 = 2020
yearn = 2070

fn = 'Regions.shp'
seas = gpd.GeoDataFrame.from_file(fn)

for model_name in ['CNRM','ECEARTH','MPI','MRI']
    rootdir = f'/Sea Ice/{model_name}/siconc'
    sic,dt,x,y = concatenate_gcm(rootdir,'siconc',year0,yearn,month)
    sic = np.where(np.isnan(sic) == False, 1,sic)
    rootdir = f'/Sea Ice/{model_name}/areacello'
    areacello = OpenAreacello(rootdir)
    sia = SIAFields(sic,areacello)
    sia_temp,regions = sea_aves(sia,seas,x,y)
    seas_sia = sia_temp
    months = ['July']
    cols = [x+str(' ')+y for (x,y) in product(months,regions)]
    df = pd.DataFrame(seas_sia.T, columns=cols)
    years = np.arange(year0,yearn,1)
    df.insert(0, ('year', ''), years)
    df.to_csv(f'{model_name}_SIA_filled_series.csv', index=False)
    print("CSV file saved successfully.")

CSV file saved successfully.
